# SQL: Advanced Concepts

-----

In this notebook, we focus on several advanced SQL concepts that build on the previous discussion of manipulating data and data structures within a relational database. Specifically, we will introduce the primary key, foreign key, table joins, and order by clause and demonstrate how they can be performed by using the SQLite relational database. 

-----

## Table of Contents

[Primary Key and Foreign Key](#Primary-Key-and-Foreign-Key)  
[Joins](#Joins)  
[Order By Clause](#Order-By-Clause)  

----

[[Back to TOC]](#Table-of-Contents)

## Primary Key and Foreign Key

### Primary Key

A primary key is a special relational database table column (or combination of columns) designated to uniquely identify all table records. Primary key must contain a unique value for each row of data.

By default, SQLite provides a rowid column for all tables. This value provides a unique identifier for each row, which can be an important constraint for relational database tables. However, this can increase the size of your table because an extra column has been added simply to provide a unique identifier. To prevent the automatic creation of this identifying column, you can specify the `WITHOUT ROWID` clause when creating a new table. 

Rather than using an automatic row identifier, you can indicate that one or more columns should be used to provide the unique identifier. This is accomplished by explicitly identifying the relevant column or columns during table creation. This can be done in two manners. First, if one specific column will be used, you can append the `PRIMARY KEY` qualifier to the relevant column when constructing the table. For example, the following example specifies that `columnOne` is an integer column that must have a non-null value that also serves as the primary key for `myTable`. 

```sql
CREATE TABLE myTable (
    columnOne NOT NULL INT PRIMARY KEY,
    ...
) ;
```

On the other hand, we can explicitly create a primary key for a table by using the primary key clause at the end of the column definitions for the relevant table, and passing in the name of the column or columns that should be used to construct the primary key. The first code snippet demonstrates the creation of a primary key for `myTable` that uses one column, while the second code snippet demonstrates using two columns.

```sql
CREATE TABLE myTable (
    columnOne NOT NULL INT,
    ...,
    PRIMARY KEY(columnOne)
) ;
```

```sql
    ...,
    PRIMARY KEY(columnOne, columnTwo)
) ;
```
Note that while SQLite does allow a primary key column to contain `NULL` values, you generally want to avoid this since it complicates unique row identification. In addition, a primary key can be an alias for a row identifier, as is the case with SQLite, but this is only true for integer primary keys. Non-integer columns are treated separately (since a mapping must be constructed between the actual values and the row identifiers). A major benefit of using primary keys is that any query involving a primary key in the search condition will improve performance since the keys are stored in a special manner to speed up queries. 

### Foreign Key

A foreign key is a field (or collection of fields) in one table that uniquely identifies a row of another table or the same table. In simpler words, the foreign key is defined in a second table, but it refers to the primary key or a unique key in the first table. It acts as a cross-reference between tables because it references the primary key of another table, thereby establishing a link between them.

The following code cells define a SQL script that will create the `myProducts` and `mySuppliers` tables, each with their own primary key. `myProducts` also has a foreign key `supplierNumber` which refers to the primary key of `mySupplier`. This foreign key constraints ensure that `supplierNumber` in `myProducts` must exist in `mySupplier`. 

-----

In [1]:
%%writefile create.sql

-- First we drop any tables if they exist

DROP TABLE IF EXISTS mySuppliers;
DROP TABLE IF EXISTS myProducts;

-- Suppliers Table
    
CREATE TABLE mySuppliers (
    supplierNumber INT NOT NULL,
    supplierName TEXT,
    PRIMARY KEY(supplierNumber)
) ;

-- Product Table
    
CREATE TABLE myProducts (
    itemNumber INT NOT NULL,
    price REAL,
    supplierNumber INT,
    stockDate TEXT,
    description TEXT,
    PRIMARY KEY(itemNumber),
    FOREIGN KEY(supplierNumber) REFERENCES mySuppliers(supplierNumber)
) ;

Overwriting create.sql


In [2]:
# Execute SQL Script to create the products and suppliers tables
!sqlite3 testdb < create.sql

In [3]:
# Display the tables contained in the test database
!sqlite3 testdb .schema

CREATE TABLE IF NOT EXISTS "myProductSupplier" (
"itemNumber" INTEGER,
  "price" REAL,
  "stockDate" TEXT,
  "description" TEXT,
  "supplierName" TEXT
);
CREATE TABLE mySuppliers (
    supplierNumber INT NOT NULL,
    supplierName TEXT,
    PRIMARY KEY(supplierNumber)
);
CREATE TABLE myProducts (
    itemNumber INT NOT NULL,
    price REAL,
    supplierNumber INT,
    stockDate TEXT,
    description TEXT,
    PRIMARY KEY(itemNumber),
    FOREIGN KEY(supplierNumber) REFERENCES mySuppliers(supplierNumber)
);


Next we insert some values to the two tables. Notice that in the below script all `supplierNumber` in `myProducts` table are in `mySuppliers`.

In [4]:
%%writefile insert.sql

-- Insert into mySuppliers

INSERT INTO mySuppliers(supplierNumber, supplierName)
VALUES (101, 'Luna Vista Limited'),
       (102, 'Mikal Arroyo Incorporated'),
       (103, 'Quiet Beach Industries') ;
    
-- Insert into myProducts    
INSERT INTO myProducts (itemNumber, price, supplierNumber, stockDate, description)
VALUES (1, 29.95, 101, '2015-02-10', 'Male bathing suit, blue'),
       (2, 49.95, 101, '2015-02-20', 'Female bathing suit, one piece, aqua'),
       (3, 9.95, 101, '2015-01-15', 'Child sand toy set'),
       (4, 24.95, 102, '2014-12-20', 'White beach towel'),
       (5, 32.95, 102,'2014-12-22', 'Blue-striped beach towel'),
       (6, 12.95, 103, '2015-03-12', 'Flip-flop'),
       (7, 34.95, 103, '2015-01-24', 'Open-toed sandal') ;

Overwriting insert.sql


In [5]:
!sqlite3 testdb < insert.sql

In [6]:
!sqlite3 testdb "SELECT * FROM mySuppliers;"

101|Luna Vista Limited
102|Mikal Arroyo Incorporated
103|Quiet Beach Industries


Now if we try to add a new supplier with existing supplierNumber, the action will fail due to unique constraint of primary key. But we can add a supplier with a new supplierNumber. We demonstrate this in the below cell.

In [7]:
#can't add new supplier with existing supplierNumber
!sqlite3 testdb "INSERT INTO mySuppliers VALUES(101, 'New Supplier');"

Error: UNIQUE constraint failed: mySuppliers.supplierNumber


In [8]:
#add new supplier with new supplierNumber
!sqlite3 testdb "INSERT INTO mySuppliers VALUES(104, 'New Supplier');"

If we try to add a new product with `supplierNumber` that is not in `mySuppliers`, we will get foreign key constraint failed error. However, SQLite by default turns foreign key constraint off. So we will not demonstrate this in this lesson. A foreign key can also indicate how row deletion or modification should be propagated across tables within the same database. As a more complex topic, however, we will not explore foreign keys in any more detail.

In the below cell, we print out the complete SQL queries that will reconstruct the whole test database with `.dump` command.

In [9]:
# Display the SQL required to reconstruct the test database
!sqlite3 testdb '.dump'

PRAGMA foreign_keys=OFF;
BEGIN TRANSACTION;
CREATE TABLE IF NOT EXISTS "myProductSupplier" (
"itemNumber" INTEGER,
  "price" REAL,
  "stockDate" TEXT,
  "description" TEXT,
  "supplierName" TEXT
);
INSERT INTO myProductSupplier VALUES(6,25.899999999999998578,'2019-04-01','Flip-flop','Quiet Beach Industries');
INSERT INTO myProductSupplier VALUES(7,34.950000000000002841,'2015-01-24','Open-toed sandal','Quiet Beach Industries');
CREATE TABLE mySuppliers (
    supplierNumber INT NOT NULL,
    supplierName TEXT,
    PRIMARY KEY(supplierNumber)
);
INSERT INTO mySuppliers VALUES(101,'Luna Vista Limited');
INSERT INTO mySuppliers VALUES(102,'Mikal Arroyo Incorporated');
INSERT INTO mySuppliers VALUES(103,'Quiet Beach Industries');
INSERT INTO mySuppliers VALUES(104,'New Supplier');
CREATE TABLE myProducts (
    itemNumber INT NOT NULL,
    price REAL,
    supplierNumber INT,
    stockDate TEXT,
    description TEXT,
    PRIMARY KEY(itemNumber),
    FOREIGN KEY(supplierNumber) REFERENCES mySup

-----
[[Back to TOC]](#Table-of-Contents)

## Joins

A properly designed relational database will likely contain many tables, that are connected by common columns (often explicitly via primary and foreign keys). To extract or update data that span multiple tables, the relevant rows within the tables must be combined together. Formally this operation is known as a **join**, since tables are temporarily joined together to perform the desired operation. In practice, you can join two tables by using an inner join or an outer join. An inner join is essentially the intersection of two tables, where the tables are matched by comparing the values of a specific column (or columns), such as `supplierNumber`. The resulting table is composed of only rows that were matched between the two tables. An outer join is more like a union of two tables, where the tables are matched by comparing the values of a specific column (or columns), but non-matching rows are still included in the resulting table and filled with NULL values as appropriate. 

The following figure illustrates four different ways that two tables: Table A and Table B, can be joined.

![Table Joins](images/joins.png)

These four join techniques are
- the **inner join** (upper left), where only those rows that share a common value in a specific column in both tables are combined in the final joined table,
- the **left outer join** (upper right), where all rows from _TABLE A_ are placed into the joined table, and an inner join is performed with _TABLE B_. Any row in the new table that does not have a match in _Table B_ is padded with null values,
- the **right outer join** (lower left), where all rows from _TABLE B_ are placed into the joined table, and an inner join is performed with _TABLE A_. Any row in the new table that does not have a match in _Table A_ is padded with null values, and
- the **full outer join** (lower right), where the combined results of a left outer join and right outer join are placed into the final joined table, with the exception that only one inner join is performed between the two tables (so no rows are double counted).

In addition, a less common join is the cross join, where every row from Table A is matched to every row from TABLE B. This is a combinatorial operation, and can quickly lead to very large tables, hence this type of operation should be used with care.

------

### INNER JOIN

We can perform an _inner join_ either by implicitly joining two tables in a `WHERE` clause or explicitly by using the `INNER JOIN` keywords. We  demonstrate an inner join in the following code cell where we perform an implicit inner join of the `myProducts` table and the `mySuppliers` table. Note, we include the SQLite command `.headers on` to display the names of the columns extracted by the `SELECT` query.

-----

In [10]:
%%writefile select.sql
.headers on

SELECT p.price, p.description, s.supplierName 
FROM myProducts AS p, mySuppliers AS s 
  WHERE p.supplierNumber = s.supplierNumber ;

Overwriting select.sql


In [11]:
# Execute SQL Script
!sqlite3 testdb < select.sql

price|description|supplierName
29.95|Male bathing suit, blue|Luna Vista Limited
49.95|Female bathing suit, one piece, aqua|Luna Vista Limited
9.95|Child sand toy set|Luna Vista Limited
24.95|White beach towel|Mikal Arroyo Incorporated
32.95|Blue-striped beach towel|Mikal Arroyo Incorporated
12.95|Flip-flop|Quiet Beach Industries
34.95|Open-toed sandal|Quiet Beach Industries


-----

This query may seem complex, due primarily to its length. But by breaking it down line-by-line we can easily follow what's happening. First, we select two columns from the `mySuppliers` table and one column from the `mySuppliers` table. Because the query joins these two tables (by using an implicit inner join), we select columns from both tables. In the FROM clause, we list both tables and provide aliases for them to simplify the full SQL statement. In the WHERE clause, we provide the logic for joining the two tables, by explicitly instructing the SQLite database to only select rows from the two tables that have matching values in their respective `supplierNumber` columns. In processing this query, SQLite first pulls all rows out of the first (left) table in the query (`myProducts`) and finds the row with a matching value in the `supplierNumber` column in the second (right) table in the query (`mySuppliers`).

-----

In the following code cell, we perform a similar query by using the `INNER JOIN` explicit join syntax. In this case, the join column is included as part of the `INNER JOIN` clause.

----

In [12]:
%%writefile join.sql
.headers on

SELECT p.itemNumber, p.price, p.stockDate, p.description, s.supplierName
    FROM myProducts as p
    INNER JOIN mySuppliers as s ON p.supplierNumber = s.supplierNumber 
    WHERE s.supplierNumber IN (101, 102);

Overwriting join.sql


In [13]:
# Execute SQL Script
!sqlite3 testdb < join.sql

itemNumber|price|stockDate|description|supplierName
1|29.95|2015-02-10|Male bathing suit, blue|Luna Vista Limited
2|49.95|2015-02-20|Female bathing suit, one piece, aqua|Luna Vista Limited
3|9.95|2015-01-15|Child sand toy set|Luna Vista Limited
4|24.95|2014-12-20|White beach towel|Mikal Arroyo Incorporated
5|32.95|2014-12-22|Blue-striped beach towel|Mikal Arroyo Incorporated


-----


### LEFT OUTER JOIN

In a left outer join, we take the **left** table in the join operation and match all rows to the rows in the **right** table. If there are no rows in the right table that match a given row in the left table, the output columns are padded with null values. This is demonstrated in the following code cell, where we write a SQL query that joins the products table to the vendor table by using an explicit `LEFT OUTER JOIN`. By definition, the final table has the same number of rows as the left table, however, in this case the final row does not have a match in the vendors table and is thus padded with a null value in the `vendorName` column.

-----

In [14]:
%%writefile left-join-p.sql
.headers on

SELECT p.itemNumber, p.price, s.supplierName
    FROM myProducts as p
    LEFT OUTER JOIN mySuppliers as s ON p.supplierNumber = s.supplierNumber 


Overwriting left-join-p.sql


In [15]:
# Execute SQL Script
!sqlite3 testdb < left-join-p.sql

itemNumber|price|supplierName
1|29.95|Luna Vista Limited
2|49.95|Luna Vista Limited
3|9.95|Luna Vista Limited
4|24.95|Mikal Arroyo Incorporated
5|32.95|Mikal Arroyo Incorporated
6|12.95|Quiet Beach Industries
7|34.95|Quiet Beach Industries


-----

### RIGHT OUTER JOIN

A right outer join is the opposite operation from the left outer join, where we start with the right table and match rows with the left table. SQLite does not support right outer joins directly, but we can switch the order of a join between two tables to simulate a right outer join. In the following code cell we define a SQL script that performs a left outer join between the vendors table and the products table. Since there are items listed in the vendor table that are not present in the products table (perhaps because the products are not currently in the store's inventory), this operation generates a table that has a row that contain null values.

-----

In [16]:
%%writefile left-join-v.sql
.headers on

SELECT p.itemNumber, p.price, s.supplierName
    FROM mySuppliers as s
    LEFT OUTER JOIN myProducts as p ON p.supplierNumber = s.supplierNumber 



Overwriting left-join-v.sql


In [17]:
# Execute SQL Script
!sqlite3 testdb < left-join-v.sql

itemNumber|price|supplierName
1|29.95|Luna Vista Limited
2|49.95|Luna Vista Limited
3|9.95|Luna Vista Limited
4|24.95|Mikal Arroyo Incorporated
5|32.95|Mikal Arroyo Incorporated
6|12.95|Quiet Beach Industries
7|34.95|Quiet Beach Industries
||New Supplier


-----

### FULL OUTER JOIN

SQLite does not support a full outer join, however, for those databases that do provide this functionality the syntax is similar to the left outer join.

### CROSS JOIN

A cross join matches every row in the left table to every row in the right table, and generates, therefore, a new table that is a combinatorial expansion of the two tables. Uses for this operation tend to be limited, in part due to the large output table size. SQLite supports this operation by using the `CROSS JOIN` clause between two tables. 

-----

-----
[[Back to TOC]](#Table-of-Contents)

## Order By Clause

In general, you can't assume that SQLite, or any database, will return rows from a query in a specific order. If the order is important, you can use the ORDER BY clause to have SQLite order the data that are returned by your query in a particular manner. Generally, you do so by specifying a column that should be used to provide the ordinal values for comparison as shown in the next two code cells. Notice how these queries use implicit joins to extract data from the products and vendors tables.

-----

In [18]:
%%writefile orderby.sql
.headers on

-- First SQL query
SELECT s.supplierNumber AS "Supplier #", supplierName as "Supplier",
    p.price AS "Price", p.itemNumber AS "Item #"
    FROM myProducts AS p, mySuppliers AS s
    WHERE p.supplierNumber = s.supplierNumber AND p.price > 20.0
    ORDER by s.supplierNumber ;

-- Display separator
.headers off
SELECT '----------------------------------------' ;
.headers on

-- Second SQL Query
SELECT s.supplierNumber AS "Supplier #", supplierName as "Supplier",
    p.price AS "Price", p.itemNumber AS "Item #"
    FROM myProducts AS p, mySuppliers AS s
    WHERE p.supplierNumber = s.supplierNumber AND p.price > 20.0
    ORDER BY s.supplierNumber ASC, p.price DESC ;

Overwriting orderby.sql


In [19]:
# Execute SQL Script
!sqlite3 testdb < orderby.sql

Supplier #|Supplier|Price|Item #
101|Luna Vista Limited|29.95|1
101|Luna Vista Limited|49.95|2
102|Mikal Arroyo Incorporated|24.95|4
102|Mikal Arroyo Incorporated|32.95|5
103|Quiet Beach Industries|34.95|7
----------------------------------------
Supplier #|Supplier|Price|Item #
101|Luna Vista Limited|49.95|2
101|Luna Vista Limited|29.95|1
102|Mikal Arroyo Incorporated|32.95|5
102|Mikal Arroyo Incorporated|24.95|4
103|Quiet Beach Industries|34.95|7


-----

In the previous example, the first query uses the ORDER BY clause to list a subset of all the rows in the table that results from joining the `mySuppliers` table to the `myProducts` table. The rows are ordered by `supplierNumber` (the subset is constructed by applying the WHERE clause). An ORDER BY clause can take either a column name, as in this example, or a column number, which is taken from the order in which the columns are listed after the SELECT keyword.

You can also specify multiple columns to use during the sorting process and even specify ASC for ascending order, which is the default, or DESC for descending order. For example, if you used the ORDER BY 1 DESC, 4 DESC clause in the first query, the query would return the same rows, but they would be ordered by using the `supplierNumber` column as the primary sort column in descending order followed by the `price` column as the secondary sort column in descending order.


## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional. The following sites allow you to try out SQL commands online.

1. The [SQLite documentation][23] provides more details on the commands presented in this notebook.
2. The [SQLite tutorial][34] provides concise descriptions of different SQL commands.
3. Another [SQLite tutorial][45] that provides a different perspective on using SQL to interact with the SQLite database.
1. [W3 Schools SQL][1], a general SQL demo site
2. [SQLZoo][2], allows you to specify the Relational Database to target

-----

[23]: https://www.sqlite.org/lang.html
[34]: https://www.tutorialspoint.com/sqlite/
[45]: http://www.sqlitetutorial.net
[1]: http://www.w3schools.com/SQL/
[2]: http://sqlzoo.net/wiki/SELECT_basics

**&copy; 2019: Gies College of Business at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode